<a href="https://colab.research.google.com/github/olaviinha/NeuralDepthPrediction/blob/main/MonocularDepthMapPrediction_timm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#<font face="Trebuchet MS" size="6">Monocular Depth Map Prediction <font color="#999" size="4">&nbsp;&nbsp;&nbsp;|&nbsp;&nbsp;&nbsp;</font><font color="#999" size="4">Timm batch processor</font><font color="#999" size="4">&nbsp;&nbsp;&nbsp;|&nbsp;&nbsp;&nbsp;</font><a href="https://github.com/olaviinha/NeuralDepthPrediction" target="_blank"><font color="#999" size="4">Github</font></a>

Generates greyscale depth map image from a monocular image using [PyTorch Image Models](https://pypi.org/project/timm/).

`image_or_dir` may be an image URL or a path to an image or a directory located in your Google Drive. If a directory path is given, all images in directory will be batch processed.

`output_dir` should be a directory located in your Google Drive. If left empty, depth map images will be saved to given input directory.

`auto_adjust` performs automated brigthness and contrast adjustment on the generated depth map image.

`skip_existing` skip file is depth map for file is already found from output directory.

In [ ]:
#@title #Setup
#@markdown This cell needs to be run only once. It will mount your Google Drive and setup prerequisites.

force_setup = False
pip_packages = 'timm'
#pip_packages = ''
# main_repository = ''

import os
from google.colab import output
import warnings
warnings.filterwarnings('ignore')
%cd /content/

# inhagcutils
if not os.path.isfile('/content/inhagcutils.ipynb') and force_setup == False:
  !pip -q install import-ipynb {pip_packages}
  !curl -s -O https://raw.githubusercontent.com/olaviinha/inhagcutils/master/inhagcutils.ipynb
import import_ipynb
from inhagcutils import *

# Mount Drive
if not os.path.isdir('/content/drive') and force_setup == False:
  from google.colab import drive
  drive.mount('/content/drive')

# Drive symlink
if not os.path.isdir('/content/mydrive') and force_setup == False:
  os.symlink('/content/drive/My Drive', '/content/mydrive')
  drive_root_set = True
drive_root = '/content/mydrive/'

dir_tmp = '/content/tmp/'
create_dirs([dir_tmp])

import cv2
import torch
import urllib.request
import matplotlib.pyplot as plt
from PIL import Image

model_type = "DPT_Large"     # MiDaS v3 - Large     (highest accuracy, slowest inference speed)
#model_type = "DPT_Hybrid"   # MiDaS v3 - Hybrid    (medium accuracy, medium inference speed)
#model_type = "MiDaS_small"  # MiDaS v2.1 - Small   (lowest accuracy, highest inference speed)

midas = torch.hub.load("intel-isl/MiDaS", model_type)

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
midas.to(device)
# midas.eval()

midas_transforms = torch.hub.load("intel-isl/MiDaS", "transforms")

if model_type == "DPT_Large" or model_type == "DPT_Hybrid":
    transform = midas_transforms.dpt_transform
else:
    transform = midas_transforms.small_transform

output.clear()
# !nvidia-smi
op(c.ok, 'Setup finished.')

In [ ]:
#@title # Do stuff
image_or_dir = "" #@param {type:"string"}
output_dir = "" #@param {type:"string"}
auto_adjust = True #@param {type:"boolean"}
skip_existing = True #@param {type:"boolean"}

if check_input_type(image_or_dir) is 'link':
  url, filename = (imgage_or_dir, path_leaf(image_or_dir))
  urllib.request.urlretrieve(url, filename)
  inputs = [filename]
else:
  input = fix_path(drive_root+image_or_dir)
  if check_input_type(input) is 'dir':
    imgs = list_images(input)
    inputs = []
    for img in imgs:
      if '_depthmap.png' not in img:
        inputs.append(img)
  elif check_input_type(input) is 'file':
    inputs = [input]

if output_dir is '' and check_input_type(image_or_dir) is not 'link':
  if os.path.isdir(input):
    output_dir = input
  if os.path.isfile(input):
    output_dir = path_dir(input)
else:
  output_dir = fix_path(drive_root+output_dir)

for i, input in enumerate(inputs):
  dm_tmp = dir_tmp+basename(input)+'_depthmap'+path_ext(input)
  dm_file = output_dir+basename(input)+'_depthmap'+path_ext(input)
  disp_dm_file = dm_file.replace(drive_root, '')
  if os.path.isfile(dm_file) and skip_existing is True:
    op(c.warn, 'File already exists, skipping', disp_dm_file)
  else:
    img = cv2.imread(input)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    input_batch = transform(img).to(device)
    with torch.no_grad():
      prediction = midas(input_batch)
      prediction = torch.nn.functional.interpolate(
          prediction.unsqueeze(1),
          size=img.shape[:2],
          mode="bicubic",
          align_corners=False,
      ).squeeze()

    output = prediction.cpu().numpy()
    if auto_adjust is True:
      output = ((output - output.min()) / (output.max()-output.min())) * 255
      output = output.astype(np.uint8)

    im = Image.fromarray(output)
    im.convert('RGB').save(dm_tmp)

    !cp {dm_tmp} {dm_file}

    if os.path.isfile(dm_file) is True:
      op(c.ok, 'Depth map saved as', disp_dm_file)
    else:
      op(c.fail, 'Error occurred.')

